In [1]:
from sklearn.externals import joblib # to save model

from P300.data import load_data

# confusion matrix
from sklearn import metrics
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html


import matplotlib.pyplot as plt
%matplotlib inline 


%load_ext autoreload
%autoreload 2

import numpy as np

In [2]:
# load classifier 
#subject='A'
#subject='B'
model_folder='model'
subject='AandB'
#clf_name = 'LR'
clf_name = 'SVM'
num_ch=64
clf_filename="{}/{}_{}_ch{}.model".format(model_folder, subject, clf_name,num_ch)
clf = joblib.load(clf_filename)
print(clf)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=5000, multi_class='ovr',
     penalty='l2', random_state=42, tol=0.0001, verbose=0)


In [3]:
# calc prob for test 
#subject = 'A'
subject = 'B'
#X_train, Y_train, Code_train = load_data(subject, 'train', 85)
X_test, Y_test, Code_test = load_data(subject, 'test', 100)

loaded:
(100, 180, 896)
(100, 180, 1)
(100, 180, 1)
stacked:
(18000, 896)
(18000,)
(18000,)


In [4]:
# test
Y_est=clf.predict(X_test)

accuracy = sum(Y_test==Y_est) / len(Y_test)
print(accuracy)

metrics.confusion_matrix(Y_test,Y_est,labels=(1,-1))

0.7613888888888889


array([[  870,  2130],
       [ 2165, 12835]])

In [5]:

# predict prob
prob = clf.predict_proba(X_test)
plt.hist(prob)

log_prob = clf.predict_log_proba(X_test)
#plt.hist(log_prob)

AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

In [ ]:
np.max(log_prob)

In [ ]:
prob.shape

In [ ]:
X_test.shape

In [ ]:
log_prob.shape

In [ ]:
plt.hist(log_prob[:,0])

In [ ]:
plt.hist(log_prob[:,1]-log_prob[:,0])

## decode after averaging over trials

In [ ]:
from P300.data import sort_by_code

XX, YY, CC= sort_by_code(X_test, Y_test, Code_test)

In [ ]:
print(XX.shape)
print(YY.shape)
print(CC.shape)

In [ ]:
#Yest=clf.predict(XX)
#clf.score(XX, YY)
log_prob = clf.predict_log_proba(XX)
log_prob=log_prob.reshape(-1,15,12,2)
log_prob.shape

In [ ]:
# accumulate log_prob over trials
log_prob_diff=np.sum(log_prob,axis=1)[:,:,1]-np.sum(log_prob,axis=1)[:,:,0]
log_prob_diff.shape

In [ ]:
YY = YY.reshape(-1,15,12)
YY.shape
#YY[1,:,1]
YY_trial=YY[:,0,:]
YY_trial.shape

In [ ]:
epoch = 0
plt.subplot(211)
plt.plot(log_prob_diff[epoch,:])
plt.subplot(212)
plt.plot(YY_trial[epoch,:])

In [ ]:
np.sum(log_prob_diff > 0 , axis=1)

In [ ]:
from P300.data import decode_rc

decoded = decode_rc(log_prob_diff)
decoded.shape
plt.plot(decoded[epoch,:])

In [ ]:
from P300.data import calc_accuracy_rc

calc_accuracy_rc(YY_trial, decoded)